In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key='',
    model="llama3-8b-8192",
    temperature=0,
)
response = llm.invoke("the first person to land on moon was...")
print(response.content)

The first person to walk on the Moon was Neil Armstrong, an American astronaut who stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.

However, it's worth noting that Armstrong was not the first person to land on the Moon. That distinction belongs to Edwin "Buzz" Aldrin, who was the second person to walk on the Moon and the first person to do so while still in the lunar module's descent stage. Armstrong and Aldrin spent about two and a half hours on the Moon's surface, collecting samples and conducting experiments.


In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/111007517923779270-software-engineer-university-graduate-2026")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Engineer, University Graduate, 2026 — Google CareersCareersCareersSkip navigation linkshomehomeHomeHomework_outlinework_outlineJobsJobsnoogler_hatnoogler_hatStudentsStudentsgooglegoogleHow we workHow we workhandymanhandymanHow we hireHow we hireperson_outlineperson_outlineYour careerYour careerhelp_outlineHelp linkfeedbackSend feedbackmore_vert HelpSend FeedbackSign inCareersCareershomeHomework_outlineJobsexpand_morenoogler_hatStudentsexpand_moregoogleHow we workexpand_morehandymanHow we hireexpand_moreperson_outlineYour careerexpand_morejob detailsarrow_backBack to jobs searchJobs search results2,830  jobs matchedSenior Software Engineer, Android PartnerWarsaw, PolandSenior Software Engineer, Google Cloud DataprocSunnyvale, CA, USASoftware Engineer III, Google CloudBengaluru, Karnataka, India; Hyderabad, Telangana, IndiaAccount Manager, Large Customer Sales (English, Thai)Bangkok, ThailandDatabase Sales Specialist Manager, Google CloudSingaporeCustomer Engineer, Migration, Go

In [3]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}

    ### INSTRUCTION:
    You are an expert job posting extractor. The above content is scraped from a job listing website.

    Your task is to extract all relevant job postings from the text and return them in **strict JSON format**.

    Each JSON object must contain the following keys:
    - "role"
    - "experience"
    - "skills"
    - "description"

    Do NOT include any introduction, explanation, or preamble.  
    Respond with **JSON only** — not Markdown, not prose, just the JSON.  
    Do NOT use triple backticks or text like "Here is the JSON:"  
    If information is missing, use an empty string.

    ### OUTPUT FORMAT:
    [
      {{
        "role": "...",
        "experience": "...",
        "skills": "...",
        "description": "..."
      }},
      ...
    ]
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data })
print(res.content)

[
  {
    "role": "Software Engineer",
    "experience": "University Graduate, 2026",
    "skills": "Unix/Linux environments, distributed systems, machine learning, information retrieval, TCP/IP, C, C++, Java, Python",
    "description": "Research, conceive, and develop software applications to extend and improve on Google's product offering. Contribute to a wide variety of projects utilizing natural language processing, artificial intelligence, data compression, machine learning, and search technologies."
  }
]


In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

# Convert list to dict (if it's a single-item list)
if isinstance(json_res, list) and len(json_res) == 1:
    json_res = json_res[0]  # Now it's a dict


In [5]:
type(json_res)

dict

In [6]:
import pandas as pd

# Define your data
data = {
    "TechStack": [
        "Python, Streamlit, LLaMA",
        "React, Node.js, MongoDB",
        "C++, Qt, Terminal Emulator",
        "Data Analysis",
        "Generative AI, LangChain, Hugging Face",
        "DSA, OOPs, Problem Solving"
    ],
    "Links": [
        "https://github.com/UpendrA-StaRK/TaxEase",
        "https://github.com/UpendrA-StaRK/TaxEase",
        "https://github.com/UpendrA-StaRK/Term_Emulator",
        "https://github.com/UpendrA-StaRK/MedEase_WebApp",
        "https://github.com/cold-email-generator",
        "https://leetcode.com/u/upendra_paluru_/"
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("myportfolio.csv", index=False)

print("CSV file 'myportfolio.csv' created successfully.")


CSV file 'myportfolio.csv' created successfully.


In [7]:
df

,TechStack,Links
0,"Python, Streamlit, LLaMA",https://github.com/UpendrA-StaRK/TaxEase
1,"React, Node.js, MongoDB",https://github.com/UpendrA-StaRK/TaxEase
2,"C++, Qt, Terminal Emulator",https://github.com/UpendrA-StaRK/Term_Emulator
3,Data Analysis,https://github.com/UpendrA-StaRK/MedEase_WebApp
4,"Generative AI, LangChain, Hugging Face",https://github.com/cold-email-generator
5,"DSA, OOPs, Problem Solving",https://leetcode.com/u/upendra_paluru_/


In [8]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["TechStack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [11]:
links = collection.query(query_texts=job['skills'],
                         n_results=2).get('metadatas',[])

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [12]:
links

[[{'links': 'https://github.com/UpendrA-StaRK/TaxEase'},
  {'links': 'https://github.com/UpendrA-StaRK/Term_Emulator'}]]

In [10]:
job = json_res
print(job['skills'])    # access the "skills" field

Unix/Linux environments, distributed systems, machine learning, information retrieval, TCP/IP, C, C++, Java, Python


In [13]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Upendra Paluru, a passionate software engineer who builds smart, tailored solutions across AI, full-stack, and automation.
    Your goal is to write a cold email expressing interest in the job above, showcasing your ability to deliver impact using
    relevant projects from your resume.

    Add the most relevant ones from the following portfolio link: {link_list}

    Keep it direct, professional, and focused on how you can help them.  
    No preamble.

    ### EMAIL (NO PREAMBLE):
    """
)
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Excited to Apply for Software Engineer Role at Google

Dear Hiring Manager,

I came across the Software Engineer role at Google and was impressed by the opportunity to contribute to a wide variety of projects utilizing natural language processing, artificial intelligence, data compression, machine learning, and search technologies. With my background in software engineering and passion for building smart solutions, I believe I would be a great fit for this role.

As a university graduate with a strong foundation in computer science, I have developed a range of skills in Unix/Linux environments, distributed systems, machine learning, information retrieval, TCP/IP, C, C++, Java, and Python. My experience with these technologies has allowed me to deliver impactful projects, such as:

* TaxEase (https://github.com/UpendrA-StaRK/TaxEase): A tax preparation platform that utilizes machine learning to provide personalized tax advice and automate the filing process. This project showca